In [66]:
import numpy as np
import pandas as pd
from scipy.io import arff
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive

In [70]:
import os
dataset_dir = '36uci'
dataset_names = os.listdir(os.path.join(os.getcwd(), dataset_dir))
datasets=[]
for i in range(0,36):
  datasets.append('/content/drive/My Drive/36uci/'+dataset_names[i]) #read datasets

In [276]:
dataset_index=23

In [ ]:
datasets[dataset_index]

In [278]:
data = arff.loadarff(datasets[dataset_index])
data = pd.DataFrame(data[0])

In [ ]:
data

In [280]:
data['Class'] = data['Class'].str.decode('utf-8') #do not add b to class while reading arff file

In [281]:
from sklearn.model_selection import train_test_split

data, test = train_test_split(data, test_size=0.25, random_state=42)

In [287]:
data=data.sample(frac=1) #shuffle dataset to build different trees

In [288]:
tree = IncrementalDTree()

In [ ]:
for i in range(data.shape[0]): #training with one online tree 
    print(f'\n\nInstance no {i}')
    tree.fit_instance(data.iloc[[i]], data.iloc[i, -1])

In [290]:
pred = tree.predict_all(test)

In [ ]:
correct=0
wrong=0
for i in range(0,len(test)):
  if str(pred[i])==str(test.iloc[i,-1]):
    correct+=1
  else:
    wrong+=1
print("Accuracy: "+str(correct/len(test))) #accuracy of one online tree

In [ ]:
preds=[]
for k in range (0,10):
  data=data.sample(frac=1) #shuffle dataset to build different trees
  tree = IncrementalDTree()
  for i in range(data.shape[0]):
    print(f'\n\nInstance no {i}')
    tree.fit_instance(data.iloc[[i]], data.iloc[i, -1]) #fit each instance iteratively
  pred = tree.predict_all(test) #predict test samples to measure ensemble performance
  preds.append(pred)

In [293]:
y=[] 
sum_of_preds=[0]*len(test)

##test evaluation;

for i in range (0,10):
  for j in range(0,len(test)):
    sum_of_preds[j]+=int(preds[i][j])

for k in range(0, len(sum_of_preds)):
  p = sum_of_preds[k]/10
  sum_of_preds[k] = round(p)

In [ ]:
true = 0 #calculating accuracy of the ensemble over the test data
false = 0

for i in range(0,len(test)):
  if (int(test.iloc[i,-1])==int(sum_of_preds[i])):
      true+=1
  else:
    false+=1

print("Ensemble accuracy: "+ str(true/(true+false)))

In [1]:
class TreeNode:
    def __init__(self, instance):
        self.instances = instance
        self.isleaf = True
        self.class_labels = instance.iloc[:, -1].mode().iloc[0]
        self.children = {}

class IncrementalDTree:
    def __init__(self):
        self.root = None

    def predict_all(self, instances): #for predicting test set
        labels = []
        for i in range(instances.shape[0]):
            try:
                labels.append(tree.predict(instances.iloc[[i]], tree.root)[0].class_labels)
            except:
                labels.append(tree.predict(instances.iloc[[i]], tree.root)[1].class_labels)
        return np.array(labels)

    def predict(self, instance, current_node): #predict to check if current instance is classified correctly, if not tree will be growed
        if current_node.isleaf == True:
            return (current_node, )

        else:
            split_attribute_val = instance[current_node.instances].iloc[0]
            try:
                return self.predict(instance.drop(current_node.instances, axis = 1), current_node.children[split_attribute_val])
            except Exception as e:
                return 'bad', current_node

    def entropy(self, columns, data, target_attributes): #calculate entropy for splitting
        frequency = {}
        entropy_data = 0.0
        i = 0
        for entry in columns:
            if (target_attributes == entry):
                break
            i = i + 1
        i = i - 1
        for entry in data:
            if (entry[i] in frequency.keys()):
                frequency[entry[i]] += 1.0
            else:
                frequency[entry[i]]  = 1.0
        for frequency in frequency.values():
            entropy_data += (-frequency/len(data)) * math.log(frequency/len(data), 2)
        return entropy_data

    def informationGain(self, columns, data, attr, target_attributes):  #calculate information gain for splitting
        frequency_of_desired_var = {}
        subset_entropy = 0.0
        i = columns.index(attr)
        for entry in data:
            if (entry[i] in frequency_of_desired_var.keys()):
                frequency_of_desired_var[entry[i]] += 1.0
            else:
                frequency_of_desired_var[entry[i]]  = 1.0

        for val in frequency_of_desired_var.keys():
            valProb        = frequency_of_desired_var[val] / sum(frequency_of_desired_var.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            subset_entropy += valProb * self.entropy(columns, dataSubset, target_attributes)

        return (self.entropy(columns, data, target_attributes) - subset_entropy)               

    def find_split(self, node):
        columns = list(node.columns)
        best = columns[0]
        maximim_gain = 0;

        for attr in columns: #for each columnn find out the information gain
            new_info_gain = self.informationGain(columns, data.values, attr, columns[-1]) 
            if new_info_gain>maximim_gain: #if the new information gain happens to be more than the current maximum, update the current max info_gain with the new gain
                maximim_gain = new_info_gain
                best = attr

        return best

    def add_node(self, instance, current_node):
        if current_node.isleaf == True:
            pass

        else:
            split_attribute_val = instance[current_node.instances].iloc[0] 
            try:
                self.add_node(instance.drop(current_node.instances, axis = 1), current_node.children[split_attribute_val])
            except Exception as e:
                new_node = TreeNode(instance)
                current_node.children[split_attribute_val] = new_node

    def split(self, node):
        split_feat = self.find_split(node.instances)
        if split_feat == None:
            return node

        else:
            for val in node.instances[split_feat]:
                temp_instances = node.instances.drop(split_feat, axis = 1)
                new_node = TreeNode(temp_instances[node.instances[split_feat] == val])
                node.children[val] = new_node
            node.instances = split_feat
            node.isleaf = False
            return node

    def fit_instance(self, instance, label): #call this to fit current instance 
        if self.root == None: #first sample of dataset, build the tree
            new_node = TreeNode(instance)
            self.root = new_node

        elif self.root != None:
            pred_node = self.predict(instance, self.root)[0]
            if pred_node != 'bad':
                pred = pred_node.class_labels
                if pred == label: #current instance is classified correctly
                    pass
                else:
                    pred_node.instances = pd.concat([pred_node.instances, instance[pred_node.instances.columns]])
                    pred_node = self.split(pred_node)
 
            else:
                self.add_node(instance, self.root)         